# Day 11-0: 텍스트 전처리 (Text Preprocessing) - 정답

**학습 시간**: 3시간

---

## 라이브러리 임포트

In [ ]:
# 기본 라이브러리
import re
from collections import Counter

# NLTK (영어 처리)
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# KoNLPy (한글 처리)
from konlpy.tag import Okt, Komoran

print("라이브러리 임포트 완료!")

---

## 🎯 실습 퀴즈 정답

---

### Q1. 기본 텍스트 정제 ⭐

**문제**: 아래 텍스트를 정제하는 함수를 작성하세요.

**요구사항**:
- 소문자 변환
- 앞뒤 공백 제거
- 중복 공백을 단일 공백으로 변환

```python
raw_text = "  Hello   WORLD!  Welcome to   NLP.  "
```

**기대 결과**: `"hello world! welcome to nlp."`

In [ ]:
# Q1 정답
raw_text = "  Hello   WORLD!  Welcome to   NLP.  "

def clean_text(text):
    """기본 텍스트 정제 함수"""
    # 1. 소문자 변환
    text = text.lower()
    
    # 2. 앞뒤 공백 제거
    text = text.strip()
    
    # 3. 중복 공백을 단일 공백으로
    text = re.sub(r'\s+', ' ', text)
    
    return text

# 실행
result = clean_text(raw_text)
print(f"원본: '{raw_text}'")
print(f"결과: '{result}'")

In [ ]:
# 테스트
assert clean_text(raw_text) == "hello world! welcome to nlp."
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
텍스트 정제는 순서가 중요합니다. 일반적으로 소문자 변환 → strip → 중복 공백 제거 순서로 진행합니다.

**핵심 개념**:
- `lower()`: 문자열을 모두 소문자로 변환
- `strip()`: 문자열 양쪽 끝의 공백 제거
- `re.sub(r'\s+', ' ', text)`: 정규표현식으로 1개 이상의 공백(`\s+`)을 단일 공백으로 치환

**대안 솔루션**:
```python
# split + join으로 중복 공백 제거
' '.join(text.lower().split())
```

**흔한 실수**:
- ❌ strip() 전에 중복 공백을 먼저 제거하면 앞뒤 공백이 남을 수 있음
- ❌ 정규표현식에서 `\s` 대신 공백 문자 ` `만 사용하면 탭, 줄바꿈이 남음

**실무 팁**:
텍스트 정제 함수를 만들어두면 데이터 전처리 파이프라인에서 재사용할 수 있습니다.

---

### Q2. 정규표현식 - 특수문자 제거 ⭐

**문제**: 정규표현식을 사용하여 특수문자를 제거하세요.

**요구사항**:
- 한글, 영어, 숫자, 공백만 남기기
- 나머지 문자는 모두 제거

```python
text = "배송 빠르네요!!! 가격 10,000원 @#$% 추천해요~~"
```

**기대 결과**: `"배송 빠르네요 가격 10000원  추천해요"`

In [ ]:
# Q2 정답
text = "배송 빠르네요!!! 가격 10,000원 @#$% 추천해요~~"

# 한글(가-힣), 영어(a-zA-Z), 숫자(0-9), 공백(\s)만 남기기
# [^...]: 대괄호 안의 문자를 제외한 모든 문자
cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)

print(f"원본: {text}")
print(f"결과: {cleaned}")

In [ ]:
# 테스트
assert cleaned == "배송 빠르네요 가격 10000원  추천해요"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
정규표현식의 부정 문자 클래스 `[^...]`를 사용하여 허용할 문자만 정의하고 나머지는 제거합니다.

**핵심 개념**:
- `[^...]`: 대괄호 안의 문자를 **제외한** 모든 문자
- `가-힣`: 한글 완성형 문자 범위 (가~힣)
- `a-zA-Z`: 영어 대소문자
- `0-9`: 숫자
- `\s`: 공백 문자 (스페이스, 탭, 줄바꿈)

**대안 솔루션**:
```python
# 특정 문자만 제거하고 싶을 때
re.sub(r'[!@#$%^&*()~]+', '', text)
```

**흔한 실수**:
- ❌ `[가-힣]` 대신 `[ㄱ-ㅎ가-힣]`로 자음/모음까지 포함하면 불완전한 글자도 남음
- ❌ 공백 `\s`를 빼먹으면 모든 공백이 제거됨

**실무 팁**:
도메인에 따라 허용할 문자를 다르게 설정하세요. 예: 이메일은 `@`, `.` 유지 필요

---

### Q3. 정규표현식 - 해시태그 추출 ⭐⭐

**문제**: SNS 게시물에서 모든 해시태그를 추출하세요.

**요구사항**:
- `re.findall()` 사용
- 해시태그 형식: `#단어`

```python
post = "오늘 날씨 좋다 #서울 #맑음 #주말나들이 점심은 #맛집 가야지"
```

**기대 결과**: `['#서울', '#맑음', '#주말나들이', '#맛집']`

In [ ]:
# Q3 정답
post = "오늘 날씨 좋다 #서울 #맑음 #주말나들이 점심은 #맛집 가야지"

# 해시태그 패턴: # + 한글/영어/숫자 (1개 이상)
hashtags = re.findall(r'#[가-힣a-zA-Z0-9]+', post)

print(f"게시물: {post}")
print(f"해시태그: {hashtags}")

In [ ]:
# 테스트
assert hashtags == ['#서울', '#맑음', '#주말나들이', '#맛집']
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`re.findall()`은 패턴과 일치하는 모든 문자열을 리스트로 반환합니다.

**핵심 개념**:
- `re.findall(pattern, string)`: 패턴과 일치하는 모든 부분 문자열 반환
- `#`: 해시태그 시작 기호 (리터럴 매칭)
- `[가-힣a-zA-Z0-9]+`: 한글/영어/숫자 1개 이상
- `+`: 앞의 패턴이 1회 이상 반복

**대안 솔루션**:
```python
# \w 사용 (단어 문자 = 영숫자 + 밑줄)
re.findall(r'#\w+', post)  # 한글은 \w에 포함됨 (Python 3)
```

**흔한 실수**:
- ❌ `+` 대신 `*`를 사용하면 `#`만 있는 것도 매칭됨
- ❌ 언더스코어(_)가 필요하면 `[가-힣a-zA-Z0-9_]+`로 수정

**실무 팁**:
실제 SNS에서는 `#` 뒤에 이모지가 올 수도 있으므로 패턴을 유연하게 조정하세요.

---

### Q4. NLTK 토큰화 ⭐⭐

**문제**: 영어 문장을 단어와 문장으로 토큰화하세요.

**요구사항**:
- `word_tokenize()`: 단어 토큰화
- `sent_tokenize()`: 문장 토큰화

```python
text = "Hello! How are you? I'm learning NLP. It's really fun."
```

In [ ]:
# Q4 정답
text = "Hello! How are you? I'm learning NLP. It's really fun."

# 단어 토큰화
word_tokens = word_tokenize(text)
print(f"단어 토큰: {word_tokens}")

# 문장 토큰화
sent_tokens = sent_tokenize(text)
print(f"\n문장 토큰: {sent_tokens}")

In [ ]:
# 테스트
assert len(word_tokens) > 10  # 10개 이상의 토큰
assert len(sent_tokens) == 4   # 4개의 문장
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
NLTK의 토큰화 함수는 단순 split()보다 정교하게 토큰화합니다. 축약형(I'm, It's)도 올바르게 분리합니다.

**핵심 개념**:
- `word_tokenize()`: Penn Treebank 스타일 토큰화
  - 축약형 분리: "I'm" → ["I", "'m"]
  - 구두점 분리: "Hello!" → ["Hello", "!"]
- `sent_tokenize()`: 문장 경계 인식
  - 마침표, 물음표, 느낌표로 문장 분리
  - "Dr."같은 약어는 분리 안 함

**대안 솔루션**:
```python
# 단순 split (정확도 낮음)
text.split()  # 구두점이 단어에 붙어있음

# 정규표현식 기반
re.findall(r"\w+|[^\w\s]", text)
```

**흔한 실수**:
- ❌ NLTK 데이터 미다운로드: `nltk.download('punkt')` 필수
- ❌ 한글에 word_tokenize() 사용: 공백 기준으로만 분리됨 (형태소 분석 필요)

**실무 팁**:
영어는 NLTK, 한글은 KoNLPy로 토큰화하세요. 혼용 텍스트는 언어 감지 후 분리 처리합니다.

---

### Q5. 불용어 제거 ⭐⭐⭐

**문제**: 영어 문장에서 불용어를 제거하세요.

**요구사항**:
- 소문자 변환 후 토큰화
- NLTK 불용어 제거
- 알파벳으로만 이루어진 단어만 유지

```python
sentence = "The movie was really amazing and I loved it very much!"
```

**기대 결과**: `['movie', 'really', 'amazing', 'loved', 'much']`

In [ ]:
# Q5 정답
sentence = "The movie was really amazing and I loved it very much!"

# 영어 불용어 목록
stop_words = set(stopwords.words('english'))

# 1. 소문자 변환
sentence_lower = sentence.lower()

# 2. 토큰화
tokens = word_tokenize(sentence_lower)

# 3. 불용어 제거 + 알파벳만 유지
filtered = [token for token in tokens 
            if token not in stop_words and token.isalpha()]

print(f"원본: {sentence}")
print(f"토큰: {tokens}")
print(f"필터: {filtered}")

In [ ]:
# 테스트
assert filtered == ['movie', 'really', 'amazing', 'loved', 'much']
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
불용어 제거는 토큰화 후에 수행합니다. set 자료형을 사용하면 O(1) 탐색으로 빠릅니다.

**핵심 개념**:
- `stopwords.words('english')`: 179개의 영어 불용어 목록
- `set()`: 빠른 멤버십 테스트 (리스트는 O(n), set은 O(1))
- `token.isalpha()`: 알파벳으로만 구성된 문자열인지 확인
- 리스트 컴프리헨션: 조건을 만족하는 토큰만 필터링

**대안 솔루션**:
```python
# 함수형 스타일
list(filter(lambda t: t not in stop_words and t.isalpha(), tokens))
```

**흔한 실수**:
- ❌ 소문자 변환 안 함: "The"는 불용어지만 "the"만 불용어 목록에 있음
- ❌ `isalpha()` 누락: 구두점("!", ".")이 결과에 포함됨
- ❌ 리스트로 불용어 탐색: 느림 (매번 전체 순회)

**실무 팁**:
'very', 'really' 같은 부사도 분석 목적에 따라 제거할 수 있습니다. 커스텀 불용어를 추가하세요.

---

### Q6. 어간 추출 vs 표제어 추출 ⭐⭐⭐

**문제**: 단어 리스트에 어간 추출과 표제어 추출을 각각 적용하세요.

**요구사항**:
- `PorterStemmer` 사용
- `WordNetLemmatizer` 사용 (동사 품사)
- 결과를 딕셔너리로 반환: `{원본: (stemmed, lemmatized)}`

```python
words = ['running', 'studies', 'played', 'better', 'happily']
```

In [ ]:
# Q6 정답
words = ['running', 'studies', 'played', 'better', 'happily']

# Stemmer와 Lemmatizer 초기화
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# 결과 딕셔너리
result = {}
for word in words:
    stemmed = stemmer.stem(word)
    lemmatized = lemmatizer.lemmatize(word, pos='v')  # 동사로 처리
    result[word] = (stemmed, lemmatized)

# 출력
print("어간 추출 vs 표제어 추출:")
print("-" * 45)
for word, (stem, lemma) in result.items():
    print(f"{word:12} → stem: {stem:10} lemma: {lemma}")

In [ ]:
# 테스트
assert result['running'][0] == 'run'      # stem
assert result['running'][1] == 'run'      # lemma
assert result['studies'][0] == 'studi'    # stem (불완전)
assert result['studies'][1] == 'study'    # lemma (완전)
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
어간 추출과 표제어 추출 모두 단어 정규화 기법이지만, 접근 방식이 다릅니다.

**핵심 개념**:
- **어간 추출 (Stemming)**: 규칙 기반으로 어미 제거
  - 장점: 빠름
  - 단점: 불완전한 결과 ("studies" → "studi")
- **표제어 추출 (Lemmatization)**: 사전 기반으로 기본형 탐색
  - 장점: 정확함 ("studies" → "study")
  - 단점: 느림, 품사 정보 필요
- `pos='v'`: 동사로 처리 (n=명사, a=형용사, r=부사)

**대안 솔루션**:
```python
# 딕셔너리 컴프리헨션
result = {
    word: (stemmer.stem(word), lemmatizer.lemmatize(word, pos='v'))
    for word in words
}
```

**흔한 실수**:
- ❌ 표제어 추출 시 품사 미지정: 기본값이 명사라서 동사가 제대로 변환 안 됨
- ❌ "better" → 표제어 추출로도 "good"으로 안 바뀜 (형용사 비교급은 WordNet에서 지원 안 함)

**실무 팁**:
검색 엔진에서는 빠른 어간 추출을, 텍스트 분석에서는 정확한 표제어 추출을 선택하세요.

---

### Q7. KoNLPy 명사 추출 ⭐⭐⭐

**문제**: 고객 리뷰에서 명사를 추출하고 빈도를 분석하세요.

**요구사항**:
- `Okt.nouns()` 사용
- 1글자 명사 제외
- 상위 5개 명사 출력

```python
reviews = [
    "배송이 빠르고 상품 품질이 좋아요",
    "가격대비 품질이 훌륭합니다",
    "배송은 빠른데 포장이 아쉬워요",
    "상품 퀄리티 최고 배송도 빠름"
]
```

In [ ]:
# Q7 정답
reviews = [
    "배송이 빠르고 상품 품질이 좋아요",
    "가격대비 품질이 훌륭합니다",
    "배송은 빠른데 포장이 아쉬워요",
    "상품 퀄리티 최고 배송도 빠름"
]

okt = Okt()

# 1. 모든 리뷰에서 명사 추출
all_nouns = []
for review in reviews:
    nouns = okt.nouns(review)
    all_nouns.extend(nouns)

# 2. 1글자 명사 제외
filtered_nouns = [noun for noun in all_nouns if len(noun) > 1]

# 3. 빈도 분석
noun_counts = Counter(filtered_nouns)

# 4. 상위 5개 출력
print("상위 5개 명사:")
for noun, count in noun_counts.most_common(5):
    print(f"  {noun}: {count}회")

In [ ]:
# 테스트
top_nouns = [noun for noun, _ in noun_counts.most_common(5)]
assert '배송' in top_nouns  # 배송이 상위에 있어야 함
assert '품질' in top_nouns  # 품질이 상위에 있어야 함
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
여러 리뷰에서 명사를 추출하고 빈도 분석으로 주요 키워드를 파악합니다.

**핵심 개념**:
- `okt.nouns()`: 문장에서 명사만 추출 (리스트 반환)
- `list.extend()`: 리스트를 확장 (append는 요소 하나 추가)
- `Counter.most_common(n)`: 빈도 상위 n개 반환
- 1글자 명사 제외: 노이즈 감소 ("것", "수", "등")

**대안 솔루션**:
```python
# 리스트 컴프리헨션 + 중첩
all_nouns = [noun for review in reviews 
             for noun in okt.nouns(review) if len(noun) > 1]
```

**흔한 실수**:
- ❌ `append()` 사용: 리스트 안에 리스트가 생김 (`extend()` 사용)
- ❌ 1글자 필터링 안 함: "것", "수" 등이 상위에 올라옴

**실무 팁**:
도메인별 불용어 목록을 만들어 추가 필터링하세요. 예: 쇼핑몰 리뷰에서 "상품", "제품"은 노이즈일 수 있음.

---

### Q8. 품사별 단어 분류 ⭐⭐⭐⭐

**문제**: 문장에서 품사별로 단어를 분류하세요.

**요구사항**:
- `Okt.pos()` 사용
- 명사(Noun), 동사(Verb), 형용사(Adjective) 분류
- 결과를 딕셔너리로 반환

```python
text = "맛있는 음식을 먹고 행복한 하루를 보냈다"
```

In [ ]:
# Q8 정답
text = "맛있는 음식을 먹고 행복한 하루를 보냈다"

okt = Okt()

# 품사 태깅 (어간 추출 적용)
pos_result = okt.pos(text, stem=True)
print(f"품사 태깅: {pos_result}")

# 품사별 분류
classified = {
    'Noun': [],
    'Verb': [],
    'Adjective': []
}

for word, pos in pos_result:
    if pos in classified:
        classified[pos].append(word)

print(f"\n분류 결과: {classified}")

In [ ]:
# 테스트
assert '음식' in classified['Noun']
assert '먹다' in classified['Verb']
assert '맛있다' in classified['Adjective']
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
품사 태깅 결과를 순회하며 원하는 품사의 단어만 수집합니다.

**핵심 개념**:
- `okt.pos()`: (단어, 품사) 튜플의 리스트 반환
- `stem=True`: 어간 추출 적용 ("먹고" → "먹다")
- Okt 품사 태그: Noun(명사), Verb(동사), Adjective(형용사), Adverb(부사) 등

**대안 솔루션**:
```python
# defaultdict 사용
from collections import defaultdict
classified = defaultdict(list)
for word, pos in pos_result:
    classified[pos].append(word)
```

**흔한 실수**:
- ❌ `stem=True` 미적용: "먹고", "보냈다" 그대로 나옴
- ❌ 품사 태그 오타: 'Noun' (O), 'noun' (X)

**실무 팁**:
감성 분석에서는 형용사가 중요하고, 키워드 추출에서는 명사가 중요합니다. 목적에 맞게 품사를 선택하세요.

---

### Q9. 한글 전처리 함수 ⭐⭐⭐⭐

**문제**: 한글 텍스트 전처리 함수를 작성하세요.

**요구사항**:
- 함수명: `clean_korean_text`
- 기능:
  1. 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
  2. 중복 공백 제거
  3. 형태소 분석 (정규화, 어간 추출)
  4. 불용어 제거 (조사, 어미)
  5. 2글자 이상 토큰만 유지
- 반환: 토큰 리스트

```python
text = "오늘 배송받은 상품이 정말 좋아요!!! 강력 추천합니다~~"
```

In [ ]:
# Q9 정답
def clean_korean_text(text, min_length=2):
    """
    한글 텍스트 전처리 함수
    
    Args:
        text (str): 원본 텍스트
        min_length (int): 최소 토큰 길이
    
    Returns:
        list: 전처리된 토큰 리스트
    """
    okt = Okt()
    
    # 1. 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    
    # 2. 중복 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 3. 형태소 분석 (정규화 + 어간 추출)
    tokens = okt.morphs(text, norm=True, stem=True)
    
    # 4. 불용어 제거
    korean_stopwords = {'은', '는', '이', '가', '을', '를', '의', '에', '에서',
                        '로', '으로', '도', '만', '과', '와', '하다', '있다', '되다',
                        '것', '수', '등', '더', '좀'}
    tokens = [t for t in tokens if t not in korean_stopwords]
    
    # 5. 최소 길이 필터링
    tokens = [t for t in tokens if len(t) >= min_length]
    
    return tokens

# 테스트
text = "오늘 배송받은 상품이 정말 좋아요!!! 강력 추천합니다~~"
result = clean_korean_text(text)

print(f"원본: {text}")
print(f"결과: {result}")

In [ ]:
# 테스트
assert '배송' in result or '받다' in result  # 핵심 단어 포함
assert '!!!' not in str(result)  # 특수문자 제거됨
assert all(len(t) >= 2 for t in result)  # 모두 2글자 이상
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
전처리 파이프라인을 함수로 캡슐화하면 재사용성이 높아집니다. 순서가 중요합니다.

**핵심 개념**:
- 전처리 순서: 특수문자 제거 → 공백 정리 → 토큰화 → 불용어 제거 → 길이 필터
- `norm=True`: 정규화 ("ㅋㅋㅋ" → "ㅋㅋ", "그래욬" → "그래요")
- `stem=True`: 어간 추출 ("좋아요" → "좋다")
- 커스텀 불용어: 조사, 의존명사 등 분석에 불필요한 단어

**대안 솔루션**:
```python
# 클래스로 구현 (상태 유지)
class KoreanPreprocessor:
    def __init__(self, stopwords=None, min_length=2):
        self.okt = Okt()
        self.stopwords = stopwords or set()
        self.min_length = min_length
    
    def preprocess(self, text):
        # ... 동일한 로직
```

**흔한 실수**:
- ❌ Okt()를 함수 내에서 매번 생성: 느림 (전역 또는 클래스 멤버로)
- ❌ 정규표현식에서 한글 범위 오타: `[가-힣]` (O), `[ㄱ-힣]` (X)

**실무 팁**:
불용어 목록은 분석 도메인에 따라 계속 업데이트하세요. 파일로 관리하면 편리합니다.

---

### Q10. 종합: 뉴스 기사 키워드 추출 ⭐⭐⭐⭐⭐

**문제**: 뉴스 기사에서 핵심 키워드를 추출하는 파이프라인을 완성하세요.

**요구사항**:
- 함수명: `extract_keywords`
- 입력: 뉴스 기사 텍스트
- 처리 과정:
  1. 특수문자 제거
  2. 형태소 분석 (Okt)
  3. 명사만 추출
  4. 1글자 명사 제외
  5. 빈도 분석
- 반환: 상위 N개 키워드 리스트 `[(키워드, 빈도), ...]`

```python
news = """
삼성전자가 새로운 갤럭시 스마트폰을 출시했다.
이번 갤럭시는 카메라 성능이 크게 향상되었으며,
배터리 용량도 늘어났다. 삼성전자 관계자는
"이번 갤럭시가 시장에서 좋은 반응을 얻을 것"이라고 밝혔다.
경쟁사인 애플의 아이폰과의 경쟁도 치열해질 전망이다.
"""
```

In [ ]:
# Q10 정답
def extract_keywords(text, top_n=10, min_length=2):
    """
    뉴스 기사에서 핵심 키워드를 추출합니다.
    
    Args:
        text (str): 뉴스 기사 텍스트
        top_n (int): 반환할 상위 키워드 수
        min_length (int): 최소 키워드 길이
    
    Returns:
        list: [(키워드, 빈도), ...] 형태의 리스트
    """
    okt = Okt()
    
    # 1. 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    
    # 2. 중복 공백 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # 3. 명사 추출 (Okt.nouns() 사용)
    nouns = okt.nouns(cleaned)
    
    # 4. 1글자 명사 제외 + 불용어 제거
    stopwords = {'것', '수', '등', '이번', '이', '그', '저'}
    filtered = [noun for noun in nouns 
                if len(noun) >= min_length and noun not in stopwords]
    
    # 5. 빈도 분석
    keyword_counts = Counter(filtered)
    
    # 6. 상위 N개 반환
    return keyword_counts.most_common(top_n)

# 테스트
news = """
삼성전자가 새로운 갤럭시 스마트폰을 출시했다.
이번 갤럭시는 카메라 성능이 크게 향상되었으며,
배터리 용량도 늘어났다. 삼성전자 관계자는
"이번 갤럭시가 시장에서 좋은 반응을 얻을 것"이라고 밝혔다.
경쟁사인 애플의 아이폰과의 경쟁도 치열해질 전망이다.
"""

keywords = extract_keywords(news, top_n=10)

print("뉴스 기사 핵심 키워드:")
print("="*30)
for keyword, count in keywords:
    print(f"  {keyword}: {count}회")

In [ ]:
# 테스트
keyword_words = [kw for kw, _ in keywords]

# 핵심 키워드가 포함되어야 함
assert '갤럭시' in keyword_words, "'갤럭시'가 키워드에 없습니다"
assert '삼성전자' in keyword_words or '삼성' in keyword_words, "'삼성전자'가 키워드에 없습니다"

# 반환 형식 검증
assert isinstance(keywords, list), "반환값이 리스트가 아닙니다"
assert isinstance(keywords[0], tuple), "키워드가 튜플이 아닙니다"
assert len(keywords[0]) == 2, "(키워드, 빈도) 형식이 아닙니다"

print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
전처리 → 명사 추출 → 필터링 → 빈도 분석의 전형적인 키워드 추출 파이프라인입니다.

**핵심 개념**:
- **전처리 파이프라인**: 순차적인 텍스트 정제 과정
- **명사 기반 키워드**: 뉴스에서 명사가 핵심 정보를 담음
- **빈도 분석**: 자주 등장하는 단어 = 중요 키워드
- **불용어 처리**: 도메인별 커스텀 불용어 적용

**대안 솔루션**:
```python
# TF-IDF 기반 (여러 문서 비교 시 유용)
from sklearn.feature_extraction.text import TfidfVectorizer

# TextRank 알고리즘 (문맥 고려)
from gensim.summarization import keywords
```

**흔한 실수**:
- ❌ 불용어 미처리: "이번", "것" 등이 상위에 올라옴
- ❌ 1글자 필터링 누락: "외", "내" 등 의미 없는 단어 포함
- ❌ 반환 형식 오류: `[(키워드, 빈도)]` 형태여야 함

**실무 팁**:
1. **TF-IDF 활용**: 단순 빈도보다 정교한 중요도 측정
2. **n-gram 고려**: "삼성전자" 같은 복합명사 추출
3. **시각화**: 워드클라우드로 결과 표현
4. **도메인 사전**: 전문 용어는 별도 사전으로 관리

In [ ]:
# 보너스: 여러 뉴스 기사에 적용
news_articles = [
    "삼성전자가 새로운 갤럭시 스마트폰을 출시했다. 카메라 성능이 향상되었다.",
    "애플이 아이폰15를 발표했다. 배터리 수명이 크게 늘었다.",
    "LG전자가 새로운 TV를 선보였다. OLED 기술을 적용했다."
]

print("뉴스별 키워드 분석:")
print("="*50)
for i, article in enumerate(news_articles, 1):
    kws = extract_keywords(article, top_n=5)
    kw_str = ', '.join([f"{kw}({cnt})" for kw, cnt in kws])
    print(f"뉴스 {i}: {kw_str}")

---

## 🎉 수고하셨습니다!

텍스트 전처리 마스터를 축하합니다! 이제 다음 단계로 넘어갑시다.

**다음 노트북**: `Day11_1_텍스트_분석_기법_Text_Analysis.ipynb`

---